# The Optimiser Interface
## Interacting with optimisers in PyBOP

This notebook demonstrates how to interact with PyBOP's optimiser classes.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Setting up the problem

The code block below sets up the model, parameter, dataset and creates the problem. For more information on this process, take a look at other notebooks in the examples directory.

In [ ]:
# Define the model
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 1})

# Define the parameters
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(0.0002, 0.0001),
        bounds=[1e-4, 1e-2],
    )
]

# Generate synthetic data
t_eval = np.arange(0, 900, 2)
sim = pybamm.Simulation(model)
sol = sim.solve(t_eval=t_eval)

# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"](t_eval),
        "Voltage [V]": sol["Voltage [V]"](t_eval),
    }
)

# Construct builder
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



## Interacting with the optimisers

Now that we have set up the required objects, we can introduce the interface for interacting with the optimisers. In this example, we will see how to use PINTS optimisers and SciPy optimisers. Here is the complete [list of optimisers](https://github.com/pybop-team/PyBOP?tab=readme-ov-file#supported-methods) supported in PyBOP.

In the following example, PINTS-based optimiser set-up is shown. Optimiser arguments can be passed using `PintsOptions` class.

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=20,
    max_iterations=50,
)
optim_one = pybop.PSO(problem, options=options)
results = optim_one.run()

Next, the use of SciPy optimisers is demonstrated. `ScipyMinimizeOptions` class is used to pass the optimiser arguments.

In [ ]:
options = pybop.ScipyMinimizeOptions(
    maxiter=50,
    method="Nelder-Mead",
    tol=1e-9,
)
optim_two = pybop.SciPyMinimize(problem, options=options)
results2 = optim_two.run()

Here is the optimisation results for Pints and SciPy optimisers.

In [ ]:
print("Estimated parameters x1:", results.x)
print("Estimated parameters x2:", results2.x)

Estimated parameters x1: [0.0003999]
Estimated parameters x2: [0.00040495]
